# Clustering Crypto

In [689]:
# Initial imports
import requests
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt
import hvplot.pandas
import plotly.express as px
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans

### Fetching Cryptocurrency Data

In [690]:
# Use the following endpoint to fetch json data
url = "https://min-api.cryptocompare.com/data/all/coinlist"

In [691]:
# Create a DataFrame 
# HINT: You will need to use the 'Data' key from the json response, then transpose the DataFrame.

In [692]:
# Alternatively, use the provided csv file:
crypto_df = pd.read_csv("crypto_data.csv")
crypto_df.set_index("Coin",inplace = True)
crypto_df.head(10)


,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
Coin,,,,,,
42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
365,365Coin,X11,True,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,True,PoW,NaN,611000
808,808,SHA-256,True,PoW/PoS,0.000000e+00,0
1337,EliteCoin,X13,True,PoW/PoS,2.927942e+10,3.14159E+11
2015,2015 coin,X11,True,PoW/PoS,NaN,0
BTC,Bitcoin,SHA-256,True,PoW,1.792718e+07,21000000
ETH,Ethereum,Ethash,True,PoW,1.076842e+08,0


### Data Preprocessing

In [693]:
# Keep only necessary columns:
# 'CoinName','Algorithm','IsTrading','ProofType','TotalCoinsMined','TotalCoinSupply'
crypto_df = crypto_df[['CoinName','Algorithm','IsTrading','ProofType','TotalCoinsMined','TotalCoinSupply']]
crypto_df.sample(10)


,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
Coin,,,,,,
CROAT,Croat,CryptoNight,True,PoW,NaN,100467441
PWR,PWR Coin,NIST5,True,PoW/PoS,1.006945e+10,0
VTR,Vtorrent,Scrypt,True,PoW/PoS,1.160472e+07,20000000
CHOOF,ChoofCoin,Scrypt,True,PoW/PoS,NaN,40000000
SSD,Sonic Screw Driver Coin,SHA-256,True,PoW/PoS,NaN,0
MUT,Mutual Coin,Scrypt,False,PoW/PoS,0.000000e+00,1000000000
QBT,Cubits,SHA-256,True,PoW/PoS,NaN,300000000
SLX,Slate,Slatechain,True,PoW,NaN,950000000
ANI,Animecoin,Quark,True,PoW,NaN,1976000000


In [694]:
# Keep only cryptocurrencies that are trading
istrading_mark = crypto_df.IsTrading != "FALSE"
crypto_df = crypto_df.loc[istrading_mark]
crypto_df.sample(10)

,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
Coin,,,,,,
WINE,WineCoin,Scrypt,True,PoW/PoS,NaN,1744000000
ERC,EuropeCoin,X11,True,PoW/PoS,1.040727e+07,384000000
KORE,Kore,X13,True,PoW/PoS,2.022465e+06,12000000
JMC,Junson Ming Chan Coin,X11,True,PoS,1.720992e+10,0
TSE,TattooCoin,Scrypt,True,PoW/PoS,NaN,500000000
KDC,Klondike Coin,Scrypt,True,PoW,NaN,20000000
BCR,BitCredit,Momentum,True,PoW,0.000000e+00,210000000
ILT,iOlite,Ethash,True,PoW,0.000000e+00,1000000000
MCAR,MasterCar,Scrypt,True,PoW/PoS,NaN,2991837


In [695]:
# Keep only cryptocurrencies with a working algorithm
algo_mark = crypto_df.Algorithm != "NaN"
crypto_df = crypto_df.loc[algo_mark]
crypto_df.sample(10)

,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
Coin,,,,,,
MN,Cryptsy Mining Contract,SHA-256,True,PoW,NaN,0
TLP,TulipCoin,Scrypt,True,PoW/PoS,NaN,5250000000
ZET,ZetaCoin,SHA-256,True,PoW,1.702044e+08,169795588
SEND,Social Send,Quark,True,PoW/PoS,NaN,150000000
CAP,BottleCaps,Scrypt,True,PoW/PoS,NaN,47433600
FOREX,FOREXCOIN,Scrypt,True,PoW,NaN,1000000000
RIDE,Ride My Car,PoS,True,PoS,0.000000e+00,100000000
CBX,CryptoBullion,Scrypt,True,PoW/PoS,1.039117e+06,1000000
GLD,GoldCoin,Scrypt,True,PoW,4.165865e+07,72245700


In [696]:
# Remove the "IsTrading" column
crypto_df = crypto_df[['CoinName','Algorithm','ProofType','TotalCoinsMined','TotalCoinSupply']]
crypto_df.head(10)

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
Coin,,,,,
42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
365,365Coin,X11,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,PoW,NaN,611000
808,808,SHA-256,PoW/PoS,0.000000e+00,0
1337,EliteCoin,X13,PoW/PoS,2.927942e+10,3.14159E+11
2015,2015 coin,X11,PoW/PoS,NaN,0
BTC,Bitcoin,SHA-256,PoW,1.792718e+07,21000000
ETH,Ethereum,Ethash,PoW,1.076842e+08,0


In [697]:
# Remove rows with at least 1 null value
crypto_df.dropna(how="all",inplace = True)
crypto_df.head(10)

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
Coin,,,,,
42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
365,365Coin,X11,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,PoW,NaN,611000
808,808,SHA-256,PoW/PoS,0.000000e+00,0
1337,EliteCoin,X13,PoW/PoS,2.927942e+10,3.14159E+11
2015,2015 coin,X11,PoW/PoS,NaN,0
BTC,Bitcoin,SHA-256,PoW,1.792718e+07,21000000
ETH,Ethereum,Ethash,PoW,1.076842e+08,0


In [698]:
# Remove rows with cryptocurrencies having no coins mined
mine_mark = (crypto_df.TotalCoinsMined != 0)
crypto_df = crypto_df.loc[mine_mark]
crypto_df.sample(10)

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
Coin,,,,,
XPO,Opair,PoS,PoS,7.403381e+07,74000000
XLC,LeviarCoin,CryptoNight,PoW,1.416180e+07,54000000
CORAL,CoralPay,X13,PoS,NaN,21000000
FLO,Flo,Scrypt,PoW,1.522707e+08,160000000
NOTE,Dnotes,Scrypt,PoS,1.746461e+08,500000000
VIVO,VIVO Coin,NeoScrypt,PoW,4.165968e+06,27000000
DPAY,DelightPay,X13,PoW/PoS,NaN,90400000
PROTON,Proton,X16R,PoS,4.403800e+06,45000000
PHR,Phreak,PoS,PoS,1.139023e+07,30000000


In [699]:
crypto_df = crypto_df.dropna(subset=['TotalCoinsMined'])
crypto_df.sample(10)

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
Coin,,,,,
KORE,Kore,X13,PoW/PoS,2.022465e+06,12000000
CAT,Catcoin,Scrypt,PoW,7.219957e+06,21000000
ETHD,Ethereum Dark,Scrypt,PoW/PoS,4.200000e+06,4200000
GNJ,GanjaCoin V2,X14,PoW/PoS,1.000000e+08,100000000
ARGUS,ArgusCoin,Scrypt,PoW/PoS,1.148324e+06,28600000
REC,Regalcoin,X11,PoW/PoS,7.262402e+06,27000000
SXC,SexCoin,Scrypt,PoW,1.289410e+08,250000000
XC,X11 Coin,X11,PoW/PoS,6.950831e+06,5500000
CPC,CapriCoin,X11,PoW/PoS,2.013643e+08,208000000


In [700]:
# Drop rows where there are 'N/A' text values
crypto_df.dropna(how='any', inplace = True)
crypto_df.head()

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
Coin,,,,,
42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
1337,EliteCoin,X13,PoW/PoS,2.927942e+10,3.14159E+11
BTC,Bitcoin,SHA-256,PoW,1.792718e+07,21000000
ETH,Ethereum,Ethash,PoW,1.076842e+08,0


In [701]:
# Store the 'CoinName'column in its own DataFrame prior to dropping it from crypto_df
coin_name_df = pd.DataFrame(data = crypto_df.CoinName)
coin_name_df.head()

,CoinName
Coin,
42,42 Coin
404,404Coin
1337,EliteCoin
BTC,Bitcoin
ETH,Ethereum


In [702]:
# Drop the 'CoinName' column since it's not going to be used on the clustering algorithm
crypto_df = crypto_df[['Algorithm','ProofType','TotalCoinsMined','TotalCoinSupply']]
crypto_df.head(10)

,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
Coin,,,,
42,Scrypt,PoW/PoS,4.199995e+01,42
404,Scrypt,PoW/PoS,1.055185e+09,532000000
1337,X13,PoW/PoS,2.927942e+10,3.14159E+11
BTC,SHA-256,PoW,1.792718e+07,21000000
ETH,Ethash,PoW,1.076842e+08,0
LTC,Scrypt,PoW,6.303924e+07,84000000
DASH,X11,PoW/PoS,9.031294e+06,22000000
XMR,CryptoNight-V7,PoW,1.720114e+07,0
ETC,Ethash,PoW,1.133597e+08,210000000


In [703]:
# Create dummy variables for text features
crypto_df_dummy = pd.get_dummies(crypto_df,columns = ["Algorithm","ProofType"])
crypto_df_dummy.sample(10)

,TotalCoinsMined,TotalCoinSupply,Algorithm_1GB AES Pattern Search,Algorithm_536,Algorithm_Argon2d,Algorithm_BLAKE256,Algorithm_Blake,Algorithm_Blake2S,Algorithm_Blake2b,Algorithm_C11,...,ProofType_PoW/PoS,ProofType_PoW/PoS,ProofType_PoW/PoW,ProofType_PoW/nPoS,ProofType_Pos,ProofType_Proof of Authority,ProofType_Proof of Trust,ProofType_TPoS,ProofType_Zero-Knowledge Proof,ProofType_dPoW/PoW
Coin,,,,,,,,,,,,,,,,,,,,,
SPD,1.125134e+07,19340594,0,0,0,0,0,0,0,1,...,1,0,0,0,0,0,0,0,0,0
GAME,6.983610e+07,84000000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
CRW,2.298693e+07,42000000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
BITG,9.803691e+06,21000000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
PLURA,5.600860e+08,1000000000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
POT,2.226772e+08,420000000,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
ABJ,9.880502e+06,30000000,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
KRONE,1.745375e+07,84000000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
BURST,1.813034e+09,2158812800,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [704]:
# Standardize data
scaler = StandardScaler()
scaler.fit(crypto_df_dummy)
crypto_df_scaled = scaler.transform(crypto_df_dummy)
print(crypto_df_scaled)

[[-0.11416166 -0.15072663 -0.04163054 ... -0.04163054 -0.04163054
  -0.04163054]
 [-0.09006124 -0.14255299 -0.04163054 ... -0.04163054 -0.04163054
  -0.04163054]
 [ 0.55458049  4.67600801 -0.04163054 ... -0.04163054 -0.04163054
  -0.04163054]
 ...
 [-0.11399501 -0.15040399 -0.04163054 ... -0.04163054 -0.04163054
  -0.04163054]
 [-0.11415873 -0.15071127 -0.04163054 ... -0.04163054 -0.04163054
  -0.04163054]
 [-0.1136708  -0.14919023 -0.04163054 ... -0.04163054 -0.04163054
  -0.04163054]]


### Reducing Dimensions Using PCA

In [705]:
# Use PCA to reduce dimensions to 3 principal components
pcs = PCA(n_components=3)
pcs_df = pcs.fit_transform(crypto_df_scaled)
pcs_df

array([[-0.32996423,  1.10585731, -0.36746763],
       [-0.31259418,  1.10621145, -0.36768349],
       [ 2.43011393,  1.67149165, -0.40509675],
       ...,
       [-0.16058558, -1.98619379,  0.38515924],
       [-0.28002384,  0.8963039 , -0.16794272],
       [-0.32895244,  1.10582983, -0.36747641]])

In [706]:
# Create a DataFrame with the principal components data

pcs_df = pd.DataFrame(data=pcs_df, columns=["PCA1","PCA2","PCA3"])
pcs_df.set_index(crypto_df.index, inplace=True)
pcs_df.head(10)

,PCA1,PCA2,PCA3
Coin,,,
42,-0.329964,1.105857,-0.367468
404,-0.312594,1.106211,-0.367683
1337,2.430114,1.671492,-0.405097
BTC,-0.134309,-1.348175,0.081060
ETH,-0.136103,-2.143883,0.216164
LTC,-0.163466,-1.065527,-0.025076
DASH,-0.376789,1.197198,-0.338823
XMR,-0.148624,-2.288701,0.189717
ETC,-0.134481,-2.143962,0.216153


### Clustering Crytocurrencies Using K-Means

#### Find the Best Value for `k` Using the Elbow Curve

In [707]:
inertia = []
k = list(range(1, 11))

# Calculate the inertia for the range of k values
for i in k:
    km = KMeans(n_clusters=i, random_state=50)
    km.fit(crypto_pca)
    inertia.append(km.inertia_)

# Create the Elbow Curve using hvPlot
elbow_data = {"k":k, "inertia":inertia}
df_elbow = pd.DataFrame(elbow_data)
df_elbow.hvplot.line(x="k", y="inertia",title="Elbow Curve", xticks=k)

:Curve   [k]   (inertia)

Running K-Means with `k=<your best value for k here>`

In [708]:
# Initialize the K-Means model
model = KMeans(n_clusters=4, random_state=55)

# Fit the model
model.fit(pcs_df)

# Predict clusters
predictions = model.predict(pcs_df)

In [709]:
crypto_df.head()

,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
Coin,,,,
42,Scrypt,PoW/PoS,4.199995e+01,42
404,Scrypt,PoW/PoS,1.055185e+09,532000000
1337,X13,PoW/PoS,2.927942e+10,3.14159E+11
BTC,SHA-256,PoW,1.792718e+07,21000000
ETH,Ethash,PoW,1.076842e+08,0


In [710]:
# Create a new DataFrame including predicted clusters and cryptocurrencies features
pcs_df["Class"] = model.labels_
pcs_df["CoinName"] = coin_name_df["CoinName"]
pcs_df["Algorithm"] = crypto_df["Algorithm"]
pcs_df["ProofType"] = crypto_df["ProofType"]
pcs_df["TotalCoinsMined"] = crypto_df["TotalCoinsMined"]
pcs_df["TotalCoinSupply"] = crypto_df["TotalCoinSupply"]
pcs_df.head()

,PCA1,PCA2,PCA3,Class,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
Coin,,,,,,,,,
42,-0.329964,1.105857,-0.367468,0,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
404,-0.312594,1.106211,-0.367683,0,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
1337,2.430114,1.671492,-0.405097,0,EliteCoin,X13,PoW/PoS,2.927942e+10,3.14159E+11
BTC,-0.134309,-1.348175,0.081060,1,Bitcoin,SHA-256,PoW,1.792718e+07,21000000
ETH,-0.136103,-2.143883,0.216164,1,Ethereum,Ethash,PoW,1.076842e+08,0


### Visualizing Results

#### 3D-Scatter with Clusters

In [720]:
# Create a 3D-Scatter with the PCA data and the clusters
fig = px.scatter_3d(
    pcs_df,
    x="PCA1",
    y="PCA2",
    z="PCA3",
    color="Class",
    symbol="Class",
    width=800,
)
fig.update_layout(legend=dict(x=0, y=1))
fig.show()

#### Table of Tradable Cryptocurrencies

In [725]:
# Table with tradable cryptos
df = pcs_df[["CoinName", "Algorithm", "ProofType", "TotalCoinSupply", "TotalCoinsMined", "Class"]]
df.hvplot.table(columns=["CoinName", "Algorithm", "ProofType", "TotalCoinSupply", "TotalCoinsMined", "Class"], width=400)

:Table   [CoinName,Algorithm,ProofType,TotalCoinSupply,TotalCoinsMined,Class]

In [731]:
# Print the total number of tradable cryptocurrencies
df.CoinName.describe()["unique"]

577

#### Scatter Plot with Tradable Cryptocurrencies

In [758]:
# Scale data to create the scatter plot
df_2 = pcs_df[[ "TotalCoinSupply", "TotalCoinsMined","CoinName"]]
df_2.head()

,TotalCoinSupply,TotalCoinsMined,CoinName
Coin,,,
42,42,4.199995e+01,42 Coin
404,532000000,1.055185e+09,404Coin
1337,3.14159E+11,2.927942e+10,EliteCoin
BTC,21000000,1.792718e+07,Bitcoin
ETH,0,1.076842e+08,Ethereum


In [759]:
# Plot the scatter with x="TotalCoinsMined" and y="TotalCoinSupply"
df_2.hvplot.scatter(
    x="TotalCoinsMined",
    y="TotalCoinSupply",
    hover_cols=["CoinName"]
)

:Scatter   [TotalCoinsMined]   (TotalCoinSupply,CoinName)